<a href="https://colab.research.google.com/github/hexabenja/DataAnalysisScraper/blob/main/DataWebScr4per.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección


Esta celda instala las bibliotecas necesarias para el web scraping con Langchain, incluidas langchain-community y beautifulsoup4.

In [14]:
%pip install -qU langchain-community beautifulsoup4
%pip install -qU langchain-unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


Esta celda importa bs4 y WebBaseLoader para cargar contenido de una página web específica (En este ejemplo catálogo CREST de sodimac).

In [34]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import re


 Inicializa el loader con la URL y obtiene el documento de forma asíncrona.

In [20]:
page_url = "https://www.sodimac.cl/sodimac-cl/marca/CREST"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs)

Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.28s/it]


Esta celda inicializa WebBaseLoader nuevamente, pero esta vez con parámetros adicionales para `bs_kwargs` y `bs_get_text_kwargs` para analizar específicamente solo el contenido de Markdown dentro de la clase `theme-doc-markdown` y para usar separadores y eliminaciones personalizados.

In [29]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(id="testId-searchResults-products", class_="jsx-636341914 search-results--products"),
    },
    bs_get_text_kwargs={"separator": ",", "strip": True},
)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.26s/it]


Esta celda elimina ciertas frases del panel de los resultados con una definición. Limpia el contenido definiendolo como cleaned_content. Usa la definición de cleaned_content para reemplazar por "". Elimina el span jsx-682461353 de las cuotas de resultado. Elimina números entre parentesís de stock disponible.

In [44]:

def parse_products_simple(content):
    # Lista de frases a eliminar
    unwanted_phrases = [
        "cuotas sin interés",
        "Agregar al Carro",
        "Por Sodimac",
        "Llega mañana",

    ]

    # Limpiar el contenido
    cleaned_content = content

    # Eliminar frases no deseadas
    for phrase in unwanted_phrases:
        cleaned_content = cleaned_content.replace(phrase, "")

    # Eliminar números entre paréntesis como (78), (13), etc.
    cleaned_content = re.sub(r',?\s*\(\d+\),?', '', cleaned_content)

    # Limpiar comas múltiples y espacios extra
    cleaned_content = re.sub(r',{2,}', ',', cleaned_content)
    cleaned_content = re.sub(r',\s*,', ',', cleaned_content)
    cleaned_content = re.sub(r'^\s*,+|,+\s*$', '', cleaned_content)
    cleaned_content = re.sub(r',?\s*jsx-682461353[^,]*,?', '', cleaned_content)

    # Separar productos por "CREST"
    products = []
    parts = cleaned_content.split("CREST")

    for part in parts:
        part = part.strip().strip(',').strip()
        if part:
            product_line = "CREST," + part
            products.append(product_line)

    return products

In [46]:
print(f"{doc.metadata}\n")
print(doc.page_content[:1000].strip())

{'source': 'https://www.sodimac.cl/sodimac-cl/marca/CREST'}

CREST,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,Por Sodimac,$  8.590,6,cuotas sin interés,(78),Agregar al Carro,CREST,Adhesivo Cerámica Piso y muro Superficie Rígido 20 kg,Por Sodimac,$  4.581,6,cuotas sin interés,Llega mañana,(13),Agregar al Carro,CREST,Adhesivo Cerámica Piso Superficie Rígido 25 kg,Por Sodimac,$  3.000,6,cuotas sin interés,(79),Agregar al Carro,CREST,Adhesivo Cerámica Muro Superficie Flexible 25 kg,Por Sodimac,$  19.290,6,cuotas sin interés,(29),Agregar al Carro,CREST,Adhesivo Cerámica Piso y muro Superficie Rígido 20 kg,Por Sodimac,$  6.785,6,cuotas sin interés,Llega mañana,(19),Agregar al Carro,CREST,Adhesivo Cerámica Piso Superficie Rígido 10 kg,Por Sodimac,$  8.394,6,cuotas sin interés,(26),Agregar al Carro,CREST,Adhesivo Cerámica Ac Muro Superficie Flexible 15 kg,Por Sodimac,$  15.432,6,cuotas sin interés,Llega mañana,(20),Agregar al Carro,CREST,Adhesivo Piedra Piso y muro Superficie Fle

In [47]:
raw_content = doc.page_content
clean_products = parse_products_simple(raw_content)

final_text = "\n".join(clean_products)
print(final_text)

CREST,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,$  8.590,6
CREST,Adhesivo Cerámica Piso y muro Superficie Rígido 20 kg,$  4.581,6
CREST,Adhesivo Cerámica Piso Superficie Rígido 25 kg,$  3.000,6
CREST,Adhesivo Cerámica Muro Superficie Flexible 25 kg,$  19.290,6
CREST,Adhesivo Cerámica Piso y muro Superficie Rígido 20 kg,$  6.785,6
CREST,Adhesivo Cerámica Piso Superficie Rígido 10 kg,$  8.394,6
CREST,Adhesivo Cerámica Ac Muro Superficie Flexible 15 kg,$  15.432,6
CREST,Adhesivo Piedra Piso y muro Superficie Flexible 20 kg,$  20.990,6
CREST,Adhesivo Multiaction Piso y muro Superficie Rígido 25 kg,$  16.988,6
CREST,Adhesivo Cerámica Ac Muro Superficie Flexible 6 kg,$  7.716,6
CREST,Frague piso/muro marfil 5 kg,$  7.580,6
CREST,Adhesivo Cerámica Ac Muro Superficie Flexible 1 kg,$  2.508,6
CREST,Adhesivo Porcelanato Muro Superficie Flexible 25 kg,$  49.990,6
CREST,Adhesivo Porcelanato Piso y muro Superficie Flexible 15 kg,$  33.290,6
CREST,Frague piso/muro chocolate 5 kg,$  7.